In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_4.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,I'q more for the driverless cars than against ...,0.0
1,Part of growing in this life is having the rig...,0.0
2,Dear Senator:\n\nI think that the Electoral Co...,0.0
3,Although it seems that the main reason for try...,0.0
4,"When asking for advice, whether at school, wor...",0.0
...,...,...
17983,Do you believe in aliens? In 1976 the Viking 1...,0.0
17984,Dear Principal\n\nThere has been many talk abo...,0.0
17985,Many people in today's world depend on cars to...,0.0
17986,"Today, there are advance technology in the soc...",0.0


In [3]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [4]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 1556])
Attention Mask size: torch.Size([17988, 1556])
Vocabulary Size: 29667
Max Sequence Length: 1556


In [5]:
model=Sequential()
input_length=1556
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=128,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)


Epoch 1/5


450/450 [==============================] - 218s 481ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 2/5
450/450 [==============================] - 217s 482ms/step - loss: 2.2316e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 218s 485ms/step - loss: 1.2861e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 218s 485ms/step - loss: 8.5085e-06 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 219s 486ms/step - loss: 6.0142e-06 - accuracy: 1.0000


In [6]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 17s 145ms/step - loss: 5.1079e-06 - accuracy: 1.0000


[5.10787367602461e-06, 1.0]

In [7]:
model.save("newmodel4.keras")